# Graph Coloring Oracle

In [1]:
from tweedledum.bool_function_compiler import BitVec

invalid = BitVec('00')
red = BitVec('01')
blue = BitVec('10')
purple = BitVec('11')

## Configurations

In [2]:
oracle_type = 'naive' # (naive, fancy)
syntehsis_method = 'xag' # (pkrm, xag)

# Optimization options
use_barenco_decomp = True
use_linear_resynth = False
use_diagonal_synth = True

## Oracles

In [3]:
from tweedledum.bool_function_compiler import BoolFunction

def naive(v0, v1, v2, v3 : BitVec(2)) -> BitVec(1):
    c_01 = (v0 != v1)
    c_123 = (v1 != v2) and (v1 != v3) and (v2 != v3)
    color_00 = (v0 != BitVec('00')) and (v1 != BitVec('00')) and (v2 != BitVec('00')) and (v3 != BitVec('00'))
    return c_01 and c_123 and color_00

def fancy(v0, v1, v2, v3 : BitVec(2)) -> BitVec(1):
    c_01 = (v0 != v1)
    c_123 = (v1 != v2) and (v1 != v3) and (v2 != v3)
    return c_01 and c_123

oracle_func = BoolFunction(naive) if oracle_type == 'naive' else BoolFunction(fancy)

In [4]:
print(f"Try 1: {oracle_func.simulate(red, red, blue, purple)}")
print(f"Try 2: {oracle_func.simulate(blue, red, blue, purple)}")
if oracle_type == 'naive':
    print(f"Try 3: {oracle_func.simulate(invalid, red, blue, purple)}")
    # When using the fancy oracle, we promise to not query it with 'invalid'
    # If we do, the result will be garbage!

Try 1: 0
Try 2: 1
Try 3: 0


In [5]:
# Using l337 programming skills:
search_space_size = 0
num_solutions = 0
for i in range(2**(2*4)):
    v = BitVec(8, i)
    if oracle_type == 'fancy' and BitVec('00') in [v[2:0], v[4:2], v[6:4], v[8:6]]:
        continue
    search_space_size += 1
    result = oracle_func.simulate(v[2:0], v[4:2], v[6:4], v[8:6])
    if result:
        num_solutions += 1
        print(v[2:0], v[4:2], v[6:4], v[8:6])
        
print(f"Search space size: {search_space_size}")
print(f"Number of solutions: {num_solutions}")

01 11 10 01
10 11 10 01
01 10 11 01
11 10 11 01
01 11 01 10
10 11 01 10
10 01 11 10
11 01 11 10
01 10 01 11
11 10 01 11
10 01 10 11
11 01 10 11
Search space size: 256
Number of solutions: 12


In [6]:
from tweedledum.synthesis import xag_synth, pkrm_synth

def use_pkrm(bool_function):
    return pkrm_synth(bool_function.truth_table(output_bit=0))

def use_xag(bool_function):
    from tweedledum.passes import parity_decomp
    circuit = xag_synth(bool_function.logic_network())
    return parity_decomp(circuit)

In [7]:
oracle_circuit = use_pkrm(oracle_func) if syntehsis_method == 'pkrm' else use_xag(oracle_func)
if use_barenco_decomp:
    from tweedledum.passes import barenco_decomp
    oracle_circuit = barenco_decomp(oracle_circuit, {'max_qubits' : 16})
print(f"Number of qubits: {oracle_circuit.num_qubits()}")
print(f"Number of instructions: {len(oracle_circuit)}")

Number of qubits: 23
Number of instructions: 61


## Initialization subcircuit

In [8]:
import numpy as np
from tweedledum.ir import Circuit
from tweedledum.operators import H, Ry, X, Measure
from tweedledum.passes import inverse

def naive_init():
    circuit = Circuit()
    qubits = [circuit.create_qubit() for i in range(8)]
    for qubit in qubits:
        circuit.apply_operator(H(), [qubit])
    return circuit
        
def fancy_init():
    theta = 2 * np.arccos(1 / np.sqrt(3))
    circuit = Circuit()
    qubits = [circuit.create_qubit() for i in range(8)]
    for i in range(0, 8, 2):
        circuit.apply_operator(Ry(theta), [qubits[i]])
        circuit.apply_operator(H(), [qubits[i], qubits[i + 1]])
        circuit.apply_operator(X(), [qubits[i + 1]])

    return circuit

init_subcircuit = naive_init() if oracle_type == 'naive' else fancy_init()
init_adj_subcircuit = inverse(init_subcircuit)

## Diffuser subcircuit

In [9]:
from tweedledum.operators import X, Z

diffuser_subcircuit = Circuit()
qubits = [diffuser_subcircuit.create_qubit() for i in range(9)] 
# Why 9? I need to account for the output qubit! Also if barenco is used
# this qubit should be left alone as it where the output rests!

diffuser_subcircuit.append(init_adj_subcircuit, qubits[0:init_subcircuit.num_qubits()], [])
for qubit in qubits[0:8]:
    diffuser_subcircuit.apply_operator(X(), [qubit])
diffuser_subcircuit.apply_operator(Z(), qubits[0:8])
for qubit in qubits[0:8]:
    diffuser_subcircuit.apply_operator(X(), [qubit])
diffuser_subcircuit.append(init_subcircuit, qubits[0:init_adj_subcircuit.num_qubits()], [])

if use_barenco_decomp:
    from tweedledum.passes import barenco_decomp
    diffuser_subcircuit = barenco_decomp(diffuser_subcircuit, {'max_qubits' : 16})

## Grover circuit

In [10]:
# Initialize
num_qubits = max(oracle_circuit.num_qubits(), diffuser_subcircuit.num_qubits())
circuit = Circuit()
qubits = [circuit.create_qubit() for i in range(num_qubits)]
cbits = [circuit.create_cbit() for i in range(8)]

circuit.apply_operator(X(), [qubits[8]])
circuit.apply_operator(H(), [qubits[8]])

circuit.append(init_subcircuit, qubits[0:init_subcircuit.num_qubits()], [])

# Grover iteration
num_iterations = int(np.floor(np.sqrt(search_space_size / num_solutions)))
for i in range(num_iterations):
    circuit.append(oracle_circuit, qubits[0:oracle_circuit.num_qubits()], [])
    circuit.append(diffuser_subcircuit, qubits[0:diffuser_subcircuit.num_qubits()], [])

if use_diagonal_synth:
    from tweedledum.ir import rotation_angle
    from tweedledum.passes import shallow_duplicate
    from tweedledum.synthesis import diagonal_synth

    new_circuit = shallow_duplicate(circuit)
    for instruction in circuit:
        if instruction.kind() == 'std.rx':
            angle = rotation_angle(instruction)
            qs = instruction.qubits()
            angles = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -angle/2, angle/2]
            new_circuit.apply_operator(H(), [qs[-1]])
            diagonal_synth(new_circuit, qs, instruction.cbits(), angles)
            new_circuit.apply_operator(H(), [qs[-1]])
        else:
            new_circuit.apply_operator(instruction)
    circuit = new_circuit

for i in range(circuit.num_cbits()):
    circuit.apply_operator(Measure(), [qubits[i]], [cbits[i]])

print(f"Number of qubits: {circuit.num_qubits()}")
print(f"Number of instructions: {len(circuit)}")

Number of qubits: 23
Number of instructions: 1802


## Local simulation

In [11]:
from tweedledum.converters_qiskit import tweedledum_to_qiskit_qc

qiskit_circuit = tweedledum_to_qiskit_qc(circuit)

In [12]:
import qiskit
from qiskit.providers.aer import QasmSimulator

# Construct an ideal simulator
sim = QasmSimulator()

# Perform an ideal simulation
result_ideal = qiskit.execute(qiskit_circuit, sim).result()
counts_ideal = result_ideal.get_counts(0)

In [13]:
# Sort count
count_sorted = sorted(counts_ideal.items(), key=lambda x:x[1], reverse=True)

for bit_string, count in count_sorted:
    v = BitVec(bit_string)
    print(v[2:0], v[4:2], v[6:4], v[8:6], f'({count})', (oracle_func.simulate(v[2:0], v[4:2], v[6:4], v[8:6])))

01 10 11 01 (94) 1
10 11 01 10 (82) 1
11 10 11 01 (81) 1
01 11 01 10 (77) 1
01 10 01 11 (76) 1
01 11 10 01 (72) 1
10 11 10 01 (72) 1
11 01 11 10 (71) 1
11 10 01 11 (71) 1
10 01 10 11 (66) 1
10 01 11 10 (61) 1
11 01 10 11 (58) 1
11 10 01 01 (3) 0
11 01 00 00 (3) 0
00 10 00 00 (3) 0
01 00 00 11 (3) 0
01 00 01 00 (2) 0
01 00 10 00 (2) 0
00 01 10 00 (2) 0
00 11 10 00 (2) 0
11 01 11 00 (2) 0
11 10 11 00 (2) 0
11 10 00 01 (2) 0
10 00 11 01 (2) 0
10 00 00 10 (2) 0
01 10 00 00 (2) 0
01 00 01 10 (2) 0
00 10 10 10 (2) 0
11 10 00 00 (2) 0
00 01 11 10 (2) 0
01 01 11 10 (2) 0
00 11 00 00 (2) 0
00 01 00 11 (2) 0
00 10 00 11 (2) 0
01 01 01 11 (2) 0
01 11 01 11 (2) 0
10 10 10 11 (2) 0
10 11 10 11 (2) 0
11 11 00 00 (2) 0
00 01 11 11 (2) 0
10 10 11 11 (2) 0
00 00 01 00 (1) 0
11 00 01 00 (1) 0
10 01 01 00 (1) 0
11 01 01 00 (1) 0
01 10 01 00 (1) 0
11 10 01 00 (1) 0
01 11 01 00 (1) 0
00 00 10 00 (1) 0
10 00 10 00 (1) 0
10 10 10 00 (1) 0
01 11 10 00 (1) 0
11 11 10 00 (1) 0
01 00 11 00 (1) 0
10 00 11 00 (1) 